In [1]:
import pandas as pd
import os

def mine_speech_action_negatives_v2():
    """
    Analyzes the full inference log to find images the model *thought*
    were 'speech_action' before the thresholding policy was applied.
    """
    # --- Configuration ---
    # Assumes your notebook is in the same directory as the log file.
    log_file = '/Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V37_20251020_112203/V37_full_inference_log.csv' 
    output_file = 'hard_negatives_for_speech_action.csv'
    
    print(f"⛏️  Mining for 'speech_action' hard negatives...")
    
    # --- Load the Data ---
    try:
        df = pd.read_csv(log_file)
        print(f"✅ Successfully loaded '{log_file}'.")
    except FileNotFoundError:
        print(f"❌ ERROR: Could not find '{log_file}'.")
        print("   Please make sure your .ipynb notebook file is in the same directory as the log file.")
        return

    # --- Check for 'top1_label' column ---
    if 'top1_label' not in df.columns:
        print("❌ ERROR: The log file is missing the 'top1_label' column.")
        print("   This likely means the log was generated with an older version of the script.")
        print("   Please ensure you have re-run the main script with the updated hierarchical_predict function.")
        return

    # --- Find Misclassifications ---
    # Look at the model's raw guess in 'top1_label' before thresholding was applied.
    misclassified_mask = (
        (df['top1_label'] == 'speech_action') & 
        (df['true_label'] != 'speech_action')
    )
    
    hard_negatives_df = df[misclassified_mask]
    
    # --- Save the Results ---
    if not hard_negatives_df.empty:
        # Define the most useful columns for manual review
        output_columns = [
            'image_path', 
            'true_label', 
            'top1_label',       # The model's raw guess
            'prediction',       # The final label after thresholding (likely 'review_lowconf')
            'confidence', 
            'entropy'
        ]
        # Ensure all desired columns exist in the DataFrame before selecting
        final_columns = [col for col in output_columns if col in hard_negatives_df.columns]
        
        hard_negatives_df[final_columns].to_csv(output_file, index=False)
        
        print(f"\n✅ Success! Found {len(hard_negatives_df)} images originally predicted as 'speech_action'.")
        print(f"   - Saved results to: '{output_file}'")
        
        print("\n   --- Breakdown of True Labels (what they were mistaken for) ---")
        print(hard_negatives_df['true_label'].value_counts().to_string())

    else:
        print("\n✅ No images were found to be misclassified as 'speech_action'.")

# --- Execute the function ---
mine_speech_action_negatives_v2()

⛏️  Mining for 'speech_action' hard negatives...
✅ Successfully loaded '/Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V37_20251020_112203/V37_full_inference_log.csv'.

✅ Success! Found 15 images originally predicted as 'speech_action'.
   - Saved results to: 'hard_negatives_for_speech_action.csv'

   --- Breakdown of True Labels (what they were mistaken for) ---
true_label
questioning                           3
concentration_anger_ambigious_sort    3
neutral_speech                        2
polite_smile_sort                     2
contempt                              1
skepticism_disbelief                  1
non-neutral_speech                    1
exertion                              1
happy_surprise                        1
